In [1]:
! pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
produtos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/produtos.csv', header=True, inferSchema=True)
vendedores = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/vendedores.csv', header=True, inferSchema=True)
clientes = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/clientes.csv', header=True, inferSchema=True)
itens_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/itens_pedido.csv', header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pagamentos_pedido.csv', header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/avaliacoes_pedido.csv', header=True, inferSchema=True)
pedidos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/pedidos.csv', header=True, inferSchema=True)

In [5]:
rdd = spark.sparkContext.parallelize(range(100))
print('Num. padrão de partições: ', rdd.getNumPartitions())

Num. padrão de partições:  2


In [7]:
rdd = spark.sparkContext.parallelize(range(100), 10)
print('Num. de partições modificado: ', rdd.getNumPartitions())

Num. de partições modificado:  10


In [10]:
reduce_rdd = rdd.coalesce(5)
print('Num. de partições após coalesce: ', reduce_rdd.getNumPartitions())

Num. de partições após coalesce:  5


In [11]:
repartitioned_rdd = rdd.repartition(20)
print('Num. de partições após repartition: ', repartitioned_rdd.getNumPartitions())

Num. de partições após repartition:  20


In [15]:
print('Núm. de partições padrão da tabela Produtos: ', produtos.rdd.getNumPartitions())

# Podemos definir a quantidade de partições já na leitura do arquivo:
produtos = spark.read.csv('drive/MyDrive/Colab Notebooks/colab_ebac/spark_data/produtos.csv', header=True, inferSchema=True).repartition(5)
print('Núm de partições definida na leitura: ', produtos.rdd.getNumPartitions())

Núm. de partições padrão da tabela Produtos:  5
Núm de partições definida na leitura:  5


In [18]:
# Ao analisar o DF, a depender da distribuição dos valores, pode ser interessante particioná-lo por um campo categórico específico
print(produtos.select('categoria_produto').distinct().count())
produtos.groupBy('categoria_produto').count().show()

# Neste caso, não faz muito sentido particionar por categoria dado a discrepância da distribuição dos valores
# mas apenas a título didático, poderíamos fazê-lo da seguinte forma:

produtos_part_categoria = produtos.repartition('categoria_produto') # basta indicar o nome do campo na repartição
print(produtos_part_categoria.rdd.getNumPartitions())
# o catalyst é inteligente o suficiente para entender que essa repatição não faz sentido, então a tabela não será repartida

74
+--------------------+-----+
|   categoria_produto|count|
+--------------------+-----+
|                 pcs|   30|
|               bebes|  919|
|           cine_foto|   28|
|               artes|   55|
|    moveis_decoracao| 2657|
|construcao_ferram...|  400|
|tablets_impressao...|    9|
|   artigos_de_festas|   26|
|fashion_roupa_mas...|   95|
|    artigos_de_natal|   65|
|          la_cuisine|   10|
|              flores|   14|
|     livros_tecnicos|  123|
|                NULL|  610|
|      telefonia_fixa|  116|
|construcao_ferram...|   91|
|          cool_stuff|  789|
|    eletrodomesticos|  370|
|   livros_importados|   31|
|            pet_shop|  719|
+--------------------+-----+
only showing top 20 rows

1


In [20]:
# mesmo assim podemos 'forçar' a repartição atribuindo um valor arbitrário
produtos_part_categoria = produtos.repartition(8, 'categoria_produto')
print(produtos_part_categoria.rdd.getNumPartitions())

# vamos ver a quantidade de registros que foi atribuída a cada partição:
particoes = produtos_part_categoria.rdd.glom().map(len).collect()
print('Número de registros por partição: ', particoes)

8
Número de registros por partição:  [6626, 1298, 5302, 6406, 5322, 3379, 692, 3926]


In [ ]:
# salvando com diferentes qtd de partições:

In [21]:
produtos.write.mode('overwrite').parquet('produtos_defaut_parquet')

In [22]:
produtos.coalesce(1).write.mode('overwrite').parquet('produtos_coalesce_1')

In [23]:
produtos.repartition(10).write.mode('overwrite').parquet('produtos_repartition_10')

In [24]:
produtos.write.mode('overwrite').partitionBy('categoria_produto').parquet('produtos_categoria_parquet')

In [25]:
spark.stop()